In [1]:
# ----------------------------------------------------------
# Audience Attention and Gaze Analysis Module
# ----------------------------------------------------------
# Description:
# This module implements real-time audience attention analysis for an
# edge AI–powered digital signage system. Viewer attention is operationalised
# through a composite Attention Indicator derived from gaze direction,
# gaze duration, and face presence stability, enabling objective measurement
# of on-screen engagement.
#
# In addition to visual analytics, ambient noise level is captured using
# an external USB microphone as a contextual environmental indicator.
# Noise measurements provide supplementary situational awareness of the
# deployment environment (e.g., background activity and crowd conditions)
# and are not used for speech recognition or audio content analysis.
#
# Functional Scope:
# - Face detection and multi-face tracking using a WiderFace-based model
#   with five-point facial landmark localisation
# - Gaze estimation based on head pose and facial landmarks to infer
#   on-screen viewing behaviour
# - Attention Indicator computation combining gaze persistence and
#   temporal face stability to mitigate transient or incidental detections
# - Ambient noise level measurement for environmental context analysis
# - Demographic estimation including age and gender classification
# - Facial emotion recognition for affective context analysis
# - Face embedding generation for short-term, privacy-preserving
#   viewer session identification
#
# System Platform:
# - Edge computing device: Raspberry Pi 5 (16 GB RAM)
# - AI accelerator: Hailo-8 (26 TOPS)
# - Image acquisition: Raspberry Pi Camera Module 3
# - Audio input: USB microphone (ambient noise level measurement only)
#
# Research Context:
# This module forms part of an AI-powered digital signage system evaluated
# through controlled in-house experimentation and field-aligned testing
# within Malaysian SME food and beverage (F&B) environments. The collected
# visual and environmental indicators support quantitative analysis of
# audience attention and engagement while adhering to privacy-by-design
# principles.
#
# File: 000_audience_gaze_analysis.ipynb
# Created: 07 February 2026
# Version: 3.2.0
# ----------------------------------------------------------

import os
import time
import json
import uuid
import logging
import threading
import numpy as np
import degirum as dg
import cv2
from picamera2 import Picamera2
from datetime import datetime
from logging.handlers import TimedRotatingFileHandler
import bme680
from scipy.optimize import linear_sum_assignment
from hailo_platform import Device
import sys
from collections import deque
import sounddevice as sd

# ----------------------------------------------------------
# Configuration
# ----------------------------------------------------------
preview_camera = True
console_output = True

# Performance — tuned for RPi 5 + Hailo-8
SKIP_FRAMES = 3
CAMERA_FPS = 10
PREVIEW_WIDTH = 640
PREVIEW_HEIGHT = 480
MAX_FACES_PER_FRAME = 3
MIN_LOOP_TIME = 0.100
DEMO_QUEUE_SIZE = 8

# Thermal management
THERMAL_THROTTLE_TEMP = 78.0
THERMAL_CRITICAL_TEMP = 82.0
THERMAL_CHECK_INTERVAL = 3.0

# Gaze detection
GAZE_YAW_THRESHOLD = 30  # ±30° horizontal
GAZE_PITCH_THRESHOLD = 20  # ±20° vertical
MIN_GAZE_DURATION = 0.5
ENGAGEMENT_TIMEOUT = 3.0

# Debug mode
GAZE_DEBUG = False  # Set True for troubleshooting

# Noise level (USB microphone)
NOISE_SAMPLE_RATE = 16000
NOISE_DURATION = 0.2
NOISE_REF_PRESSURE = 20e-6
NOISE_READ_INTERVAL = 5.0

# Logging
LOG_INTERVAL = 5.0
OUTPUT_DIR = "../output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# DeGirum / Hailo
inference_host_address = "@local"
zoo_url = "../models"
token = ""
device_type = "HAILORT/HAILO8"

widerface_model_name = "yolov8n_relu6_widerface_kpts--640x640_quant_hailort_hailo8_1"
face_embed_model_name = "arcface_mobilefacenet--112x112_quant_hailort_hailo8_1"
age_model_name = "yolov8n_relu6_age--256x256_quant_hailort_hailo8_1"
gender_model_name = "yolov8n_relu6_fairface_gender--256x256_quant_hailort_hailo8_1"
emotion_model_name = "emotion_recognition_fer2013--64x64_quant_hailort_multidevice_1"

EMB_DIM = 128

viewer_profiles = {}
_ZERO_EMB = np.zeros(EMB_DIM, dtype=np.float32)

# ----------------------------------------------------------
# Logging
# ----------------------------------------------------------
LOG_DIR = "../logs"
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = f"{LOG_DIR}/audience_analysis_live.log"


class FlushingTimedRotatingFileHandler(TimedRotatingFileHandler):
    """TimedRotatingFileHandler that flushes + fsync after every emit."""
    def emit(self, record):
        super().emit(record)
        try:
            self.flush()
            if hasattr(self.stream, 'fileno'):
                os.fsync(self.stream.fileno())
        except Exception:
            pass


logger = logging.getLogger("audience_analysis")
logger.setLevel(logging.INFO)
logger.handlers.clear()
logger.propagate = False

_fmt = logging.Formatter("%(asctime)s [%(levelname)s] %(message)s")

_fh = FlushingTimedRotatingFileHandler(
    LOG_FILE, when="W0", interval=1, backupCount=100, utc=True
)
_fh.setFormatter(_fmt)
logger.addHandler(_fh)

if console_output:
    _ch = logging.StreamHandler(sys.stdout)
    _ch.setFormatter(_fmt)
    logger.addHandler(_ch)

logger.info(f"=== Log initialized: {os.path.abspath(LOG_FILE)} ===")
if os.path.exists(LOG_FILE):
    logger.info(f"Log file size: {os.path.getsize(LOG_FILE)} bytes")


def log_gaze_event(data):
    """Log structured JSON event."""
    try:
        logger.info(f"GAZE_EVENT: {json.dumps(data)}")
    except (TypeError, ValueError) as e:
        logger.warning(f"Serialize error: {e}")


def handle_uncaught_exceptions(exc_type, exc_value, exc_tb):
    if issubclass(exc_type, KeyboardInterrupt):
        sys.__excepthook__(exc_type, exc_value, exc_tb)
        return
    logger.critical("Uncaught Exception", exc_info=(exc_type, exc_value, exc_tb))

sys.excepthook = handle_uncaught_exceptions


# ----------------------------------------------------------
# CPU Temperature
# ----------------------------------------------------------
def get_cpu_temp():
    try:
        with open("/sys/class/thermal/thermal_zone0/temp") as f:
            return float(f.read().strip()) / 1000.0
    except Exception:
        return 0.0


# ----------------------------------------------------------
# BME688 Environmental Sensor
# ----------------------------------------------------------
bme_sensor = None
bme_data_cache = {
    "temp_c": 0.0, "humidity": 0.0,
    "pressure_hPa": 0.0, "gas_resistance_ohms": 0.0
}
bme_last_read = 0.0


def set_bme688_sensor(s):
    s.set_humidity_oversample(bme680.OS_2X)
    s.set_pressure_oversample(bme680.OS_4X)
    s.set_temperature_oversample(bme680.OS_8X)
    s.set_filter(bme680.FILTER_SIZE_3)
    s.set_gas_status(bme680.ENABLE_GAS_MEAS)
    s.set_gas_heater_temperature(320)
    s.set_gas_heater_duration(150)
    s.select_gas_heater_profile(0)


BME_READ_INTERVAL = 10.0

try:
    bme_sensor = bme680.BME680(bme680.I2C_ADDR_PRIMARY)
    set_bme688_sensor(bme_sensor)
    logger.info("BME688 sensor initialized")
except (RuntimeError, IOError):
    try:
        bme_sensor = bme680.BME680(bme680.I2C_ADDR_SECONDARY)
        set_bme688_sensor(bme_sensor)
        logger.info("BME688 sensor initialized (secondary)")
    except Exception as e:
        logger.warning(f"BME688 not available: {e}")


def read_bme688_data():
    global bme_data_cache, bme_last_read
    if bme_sensor is None:
        return bme_data_cache
    now = time.time()
    if now - bme_last_read < BME_READ_INTERVAL:
        return bme_data_cache
    try:
        if bme_sensor.get_sensor_data():
            bme_data_cache = {
                "temp_c": round(bme_sensor.data.temperature, 2),
                "humidity": round(bme_sensor.data.humidity, 2),
                "pressure_hPa": round(bme_sensor.data.pressure, 2),
                "gas_resistance_ohms": round(bme_sensor.data.gas_resistance, 2)
            }
            bme_last_read = now
    except Exception:
        pass
    return bme_data_cache


# ----------------------------------------------------------
# Ambient Noise Level
# ----------------------------------------------------------
_noise_cache = {"db": None, "ts": 0.0}

def read_noise_level_db():
    now = time.time()
    if now - _noise_cache["ts"] < NOISE_READ_INTERVAL:
        return _noise_cache["db"]

    try:
        audio = sd.rec(
            int(NOISE_SAMPLE_RATE * NOISE_DURATION),
            samplerate=NOISE_SAMPLE_RATE,
            channels=1,
            dtype='float32',
            blocking=True
        )

        rms = np.sqrt(np.mean(np.square(audio)))
        if rms > 0:
            db = 20 * np.log10(rms / NOISE_REF_PRESSURE)
            db = round(float(db), 1)
        else:
            db = 0.0

        _noise_cache.update({"db": db, "ts": now})
        return db

    except Exception as e:
        logger.debug(f"Noise read failed: {e}")
        return _noise_cache["db"]


# ----------------------------------------------------------
# Threaded Camera Capture
# ----------------------------------------------------------
class ThreadedCamera:
    def __init__(self, fps=10, width=640, height=480):
        self.picam2 = Picamera2()
        config = self.picam2.create_preview_configuration(
            main={"format": "RGB888", "size": (width, height)},
            controls={"FrameRate": fps}
        )
        self.picam2.configure(config)
        self.picam2.start(show_preview=False)
        time.sleep(1.0)

        self._frame = None
        self._lock = threading.Lock()
        self._running = True
        self._interval = 1.0 / fps

        self._thread = threading.Thread(target=self._capture_loop, daemon=True, name="cam-capture")
        self._thread.start()
        logger.info(f"Threaded camera started at {width}x{height} @ {fps}fps")

    def _capture_loop(self):
        while self._running:
            try:
                start = time.time()
                frame = self.picam2.capture_array()
                with self._lock:
                    self._frame = frame
                elapsed = time.time() - start
                sleep = max(0, self._interval - elapsed)
                if sleep > 0:
                    time.sleep(sleep)
            except Exception:
                if self._running:
                    time.sleep(0.1)

    def read(self):
        with self._lock:
            return self._frame

    def stop(self):
        self._running = False
        self._thread.join(timeout=2.0)
        try:
            self.picam2.stop()
        except Exception:
            pass


# ----------------------------------------------------------
# Utilities
# ----------------------------------------------------------
def cosine_distance(a, b):
    d = np.dot(a, b)
    n = np.linalg.norm(a) * np.linalg.norm(b)
    return 1.0 - (d / n) if n > 0 else 1.0


def parse_keypoints(kpts_data):
    if kpts_data is None:
        return None
    try:
        if isinstance(kpts_data, np.ndarray):
            if kpts_data.shape == (5, 2):
                return kpts_data.astype(np.float32)
            if kpts_data.ndim == 1 and kpts_data.size == 10:
                return kpts_data.reshape(5, 2).astype(np.float32)
        elif isinstance(kpts_data, (list, tuple)):
            a = np.array(kpts_data, dtype=np.float32)
            if a.size == 10:
                return a.reshape(5, 2)
            if a.shape == (5, 2):
                return a
        elif isinstance(kpts_data, dict):
            if 'data' in kpts_data:
                return parse_keypoints(kpts_data['data'])
            if 'x' in kpts_data and 'y' in kpts_data:
                x = np.array(kpts_data['x'], dtype=np.float32)
                y = np.array(kpts_data['y'], dtype=np.float32)
                if len(x) == 5 and len(y) == 5:
                    return np.column_stack([x, y])
        elif isinstance(kpts_data, list) and kpts_data and isinstance(kpts_data[0], dict):
            c = []
            for pt in kpts_data[:5]:
                if 'x' in pt and 'y' in pt:
                    c.append([float(pt['x']), float(pt['y'])])
            if len(c) == 5:
                return np.array(c, dtype=np.float32)
    except Exception:
        pass
    return None


def extract_embedding(emb_data):
    try:
        if isinstance(emb_data, np.ndarray):
            return emb_data.flatten().astype(np.float32)
        if isinstance(emb_data, dict):
            for k in ("data", "embedding", "vector"):
                v = emb_data.get(k)
                if v is not None:
                    if isinstance(v, np.ndarray):
                        return v.flatten().astype(np.float32)
                    if isinstance(v, list) and v:
                        if isinstance(v[0], list):
                            v = v[0]
                        return np.array(v, dtype=np.float32)
        if isinstance(emb_data, list) and emb_data:
            if isinstance(emb_data[0], list):
                emb_data = emb_data[0]
            return np.array(emb_data, dtype=np.float32)
    except Exception:
        pass
    return _ZERO_EMB.copy()


# ----------------------------------------------------------
# Head Pose Estimation
# ----------------------------------------------------------
_PTS3D = np.array([
    [-30, 35, 30], [30, 35, 30], [0, 0, 60],
    [-25, -35, 20], [25, -35, 20]
], dtype=np.float32)
_DIST = np.zeros(5, dtype=np.float32)
_f = 0.9 * PREVIEW_WIDTH
_K = np.array([[_f, 0, PREVIEW_WIDTH / 2],
               [0, _f, PREVIEW_HEIGHT / 2],
               [0, 0, 1]], dtype=np.float32)


def head_pose_from_5pts(pts2d):
    """
    Estimate head pose from 5 facial keypoints using solvePnP.
    Returns (yaw, pitch, roll) in degrees or (None, None, None) on failure.
    """
    if pts2d is None:
        if GAZE_DEBUG:
            logger.debug("head_pose: pts2d is None")
        return None, None, None
    try:
        if not isinstance(pts2d, np.ndarray) or pts2d.shape != (5, 2):
            if GAZE_DEBUG:
                logger.debug(f"head_pose: invalid shape - type={type(pts2d)} shape={pts2d.shape if isinstance(pts2d, np.ndarray) else 'N/A'}")
            return None, None, None
        
        # Check if keypoints are reasonable (within frame bounds)
        if np.any(pts2d < 0) or np.any(pts2d[:, 0] > PREVIEW_WIDTH) or np.any(pts2d[:, 1] > PREVIEW_HEIGHT):
            if GAZE_DEBUG:
                logger.debug(f"head_pose: keypoints out of bounds - {pts2d}")
            return None, None, None
        
        ok, rvec, tvec = cv2.solvePnP(_PTS3D, pts2d, _K, _DIST, flags=cv2.SOLVEPNP_ITERATIVE)
        
        if not ok:
            if GAZE_DEBUG:
                logger.debug("head_pose: solvePnP returned ok=False")
            return None, None, None
        
        R, _ = cv2.Rodrigues(rvec)
        sy = np.sqrt(R[0, 0]**2 + R[1, 0]**2)
        
        yaw = np.degrees(np.arctan2(R[1, 0], R[0, 0]))
        pitch = np.degrees(np.arctan2(-R[2, 0], sy))
        roll = np.degrees(np.arctan2(R[2, 1], R[2, 2]))
        
        if GAZE_DEBUG:
            logger.debug(f"head_pose: SUCCESS - yaw={yaw:.1f}° pitch={pitch:.1f}° roll={roll:.1f}°")
        
        return yaw, pitch, roll
        
    except Exception as e:
        if GAZE_DEBUG:
            logger.debug(f"head_pose: exception - {e}")
        return None, None, None


def fallback_gaze_estimation(bbox):
    """
    Fallback method: estimate gaze based on face position in frame.
    Assumes face in center of frame is looking at screen.
    """
    try:
        x1, y1, x2, y2 = bbox
        face_center_x = (x1 + x2) / 2.0
        face_center_y = (y1 + y2) / 2.0
        
        # Normalize to [-1, 1]
        rel_x = (face_center_x - PREVIEW_WIDTH/2) / (PREVIEW_WIDTH/2)
        rel_y = (face_center_y - PREVIEW_HEIGHT/2) / (PREVIEW_HEIGHT/2)
        
        # Convert to approximate angles
        yaw = rel_x * 40.0  # Max ±40 degrees
        pitch = rel_y * 25.0  # Max ±25 degrees
        
        if GAZE_DEBUG:
            logger.debug(f"fallback_gaze: face_center=({face_center_x:.0f},{face_center_y:.0f}) -> yaw={yaw:.1f}° pitch={pitch:.1f}°")
        
        return yaw, pitch
        
    except Exception as e:
        if GAZE_DEBUG:
            logger.debug(f"fallback_gaze: exception - {e}")
        return None, None


def is_gazing_at_screen(pts2d, bbox=None):
    """
    Determine if viewer is gazing at screen.
    Returns (is_gazing, yaw, pitch)
    """
    # Try method 1: Proper head pose estimation
    yaw, pitch, _ = head_pose_from_5pts(pts2d)
    
    # If method 1 failed, try fallback
    if yaw is None and bbox is not None:
        if GAZE_DEBUG:
            logger.debug("Using fallback gaze estimation (bbox position)")
        yaw, pitch = fallback_gaze_estimation(bbox)
    
    # If both methods failed
    if yaw is None:
        if GAZE_DEBUG:
            logger.debug("All gaze estimation methods failed")
        return False, None, None
    
    # Check if within gaze cone
    is_gazing = (abs(yaw) <= GAZE_YAW_THRESHOLD and abs(pitch) <= GAZE_PITCH_THRESHOLD)
    
    if GAZE_DEBUG:
        status = "✓ GAZING" if is_gazing else "✗ not gazing"
        logger.info(f"{status}: yaw={yaw:+.1f}° pitch={pitch:+.1f}° (thresholds: ±{GAZE_YAW_THRESHOLD}° yaw, ±{GAZE_PITCH_THRESHOLD}° pitch)")
    
    return is_gazing, yaw, pitch


# ----------------------------------------------------------
# Viewer Tracker
# ----------------------------------------------------------
class ViewerTracker:
    def __init__(self, iou_thr=0.3, emb_thr=0.5, timeout=5.0):
        self.iou_thr = iou_thr
        self.emb_thr = emb_thr
        self.timeout = timeout
        self.tracks = {}

    @staticmethod
    def _iou(a, b):
        xA, yA = max(a[0], b[0]), max(a[1], b[1])
        xB, yB = min(a[2], b[2]), min(a[3], b[3])
        inter = max(0, xB - xA) * max(0, yB - yA)
        union = (a[2]-a[0])*(a[3]-a[1]) + (b[2]-b[0])*(b[3]-b[1]) - inter
        return inter / union if union > 0 else 0

    def _clean(self):
        now = time.time()
        stale = [k for k, v in self.tracks.items() if now - v['ts'] > self.timeout]
        for k in stale:
            del self.tracks[k]

    def update(self, bboxes, embs):
        self._clean()
        T = list(self.tracks.keys())
        N, M = len(T), len(bboxes)
        now = time.time()

        if N == 0:
            out = []
            for bb, emb in zip(bboxes, embs):
                nid = uuid.uuid4().hex[:8]
                self.tracks[nid] = {'bbox': bb, 'emb': emb, 'ts': now}
                out.append((nid, True))
            return out

        cost = np.zeros((N, M), dtype=np.float32)
        for i, tid in enumerate(T):
            t = self.tracks[tid]
            for j in range(M):
                cost[i, j] = 0.4 * (1 - self._iou(t['bbox'], bboxes[j])) + 0.6 * cosine_distance(t['emb'], embs[j])

        rows, cols = linear_sum_assignment(cost)
        results = [None] * M

        for r, c in zip(rows, cols):
            tid = T[r]
            t = self.tracks[tid]
            if self._iou(t['bbox'], bboxes[c]) >= self.iou_thr or cosine_distance(t['emb'], embs[c]) <= self.emb_thr:
                t.update({'bbox': bboxes[c], 'emb': embs[c], 'ts': now})
                results[c] = (tid, False)

        for j in range(M):
            if results[j] is None:
                nid = uuid.uuid4().hex[:8]
                self.tracks[nid] = {'bbox': bboxes[j], 'emb': embs[j], 'ts': now}
                results[j] = (nid, True)

        return results


# ----------------------------------------------------------
# Gaze Session Manager - FIXED VERSION
# ----------------------------------------------------------
class GazeSessionManager:
    def __init__(self):
        self.sessions = {}

    def update(self, vid, is_gazing, ts):
        """
        Update gaze session. Returns (total_gaze, continuous_gaze, is_new_gaze)
        """
        if vid not in self.sessions:
            self.sessions[vid] = {
                'total': 0.0, 'start': None, 'prev': ts,
                'sess_start': ts, 'last_seen': ts, 'count': 0, 'appearances': 0
            }
        s = self.sessions[vid]
        s['appearances'] += 1
        s['prev'] = ts
        s['last_seen'] = ts

        continuous = 0.0
        is_new_gaze = False  # Flag for new gaze start
        
        if is_gazing:
            if s['start'] is None:
                # NEW GAZE SESSION STARTED
                s['start'] = ts
                s['count'] += 1
                is_new_gaze = True  # Set flag
            continuous = ts - s['start']
        else:
            if s['start'] is not None:
                # GAZE SESSION ENDED
                dur = ts - s['start']
                if dur >= MIN_GAZE_DURATION:
                    s['total'] += dur
                s['start'] = None
        
        return s['total'], continuous, is_new_gaze

    def end_session(self, vid):
        """End a session and return statistics"""
        if vid not in self.sessions:
            return None
        s = self.sessions[vid]
        now = time.time()
        
        # Add any ongoing gaze time
        if s['start'] is not None:
            dur = now - s['start']
            if dur >= MIN_GAZE_DURATION:
                s['total'] += dur
        
        elapsed = now - s['sess_start']
        
        # Calculate engagement with cap at 100%
        raw_engagement = s['total'] / elapsed if elapsed > 0 else 0
        capped_engagement = min(raw_engagement, 1.0)
        
        stats = {
            'viewer_id': vid,
            'total_gaze_time': round(s['total'], 2),
            'gaze_count': s['count'],
            'session_duration': round(elapsed, 2),
            'total_appearances': s['appearances'],
            'engagement_rate': round(capped_engagement, 4),
            'raw_engagement_rate': round(raw_engagement, 4)
        }
        del self.sessions[vid]
        return stats

    def cleanup_stale(self):
        """Clean up sessions that haven't been seen recently"""
        now = time.time()
        stale = [v for v, s in self.sessions.items() 
                 if now - s['last_seen'] > ENGAGEMENT_TIMEOUT]
        ended = []
        for vid in stale:
            st = self.end_session(vid)
            if st:
                ended.append(st)
        return ended


# ----------------------------------------------------------
# Demographics Worker
# ----------------------------------------------------------
class DemographicsWorker:
    def __init__(self, age_mdl, gender_mdl, emotion_mdl, queue_size=8):
        self._age_mdl = age_mdl
        self._gender_mdl = gender_mdl
        self._emotion_mdl = emotion_mdl
        self._queue = deque(maxlen=queue_size)
        self._running = True
        self._thread = threading.Thread(target=self._run, daemon=True, name="demo-worker")
        self._thread.start()
        logger.info(f"Demographics worker started (queue_size={queue_size})")

    def submit(self, vid, crop):
        self._queue.append((vid, crop))

    def _run(self):
        while self._running:
            if not self._queue:
                time.sleep(0.05)
                continue
            try:
                vid, crop = self._queue.popleft()
                self._process(vid, crop)
            except Exception:
                logger.debug(f"Demographics failed for {vid}", exc_info=True)
            time.sleep(0.01)

    def _process(self, vid, crop):
        if crop is None or crop.size == 0:
            return

        age_val, gen_val, emo_val = 0, "", ""

        try:
            r = self._age_mdl.predict(crop)
            if hasattr(r, 'results') and r.results:
                d = r.results[0]
                raw = d.get("score", 0) if isinstance(d, dict) else getattr(d, "score", 0)
                age_val = round(raw) if raw else 0
        except Exception:
            pass

        try:
            r = self._gender_mdl.predict(crop)
            if hasattr(r, 'results') and r.results:
                d = r.results[0]
                gen_val = d.get("label", "") if isinstance(d, dict) else getattr(d, "label", "")
        except Exception:
            pass

        try:
            r = self._emotion_mdl.predict(crop)
            if hasattr(r, 'results') and r.results:
                d = r.results[0]
                emo_val = d.get("label", "") if isinstance(d, dict) else getattr(d, "label", "")
        except Exception:
            pass

        if vid in viewer_profiles:
            if age_val > 0:
                viewer_profiles[vid]['age'] = age_val
            if gen_val:
                viewer_profiles[vid]['gender'] = gen_val
            if emo_val:
                viewer_profiles[vid]['emotions'][emo_val] = viewer_profiles[vid]['emotions'].get(emo_val, 0) + 1
            logger.debug(f"Demo result: {vid} age={age_val} gender={gen_val} emotion={emo_val}")

    def stop(self):
        self._running = False
        self._thread.join(timeout=3.0)


# ----------------------------------------------------------
# Visualization
# ----------------------------------------------------------
def draw_overlay(img, faces):
    for f in faces:
        bb = f.get("bbox")
        if not bb:
            continue
        x1, y1, x2, y2 = map(int, bb)
        
        gazing = f.get("is_gazing", False)
        color = (0, 255, 0) if gazing else (0, 165, 255)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        
        vid = f.get("viewer_id", "?")[:6]
        age = f.get("age_est", 0)
        gender = f.get("gender", "")[:1]
        gaze_dur = f.get("gaze_duration", 0.0)
        yaw = f.get("yaw")
        pitch = f.get("pitch")
        
        label_parts = [f"ID:{vid}"]
        if age > 0:
            label_parts.append(f"{gender}{age}")
        if gazing:
            label_parts.append(f"GAZE:{gaze_dur:.1f}s")
        if yaw is not None and pitch is not None:
            label_parts.append(f"Y{yaw:+.0f}°P{pitch:+.0f}°")
        
        label = " ".join(label_parts)
        cv2.putText(img, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)
        
        kpts = f.get("kpts")
        if kpts is not None and isinstance(kpts, np.ndarray) and kpts.shape == (5, 2):
            for kp in kpts:
                cv2.circle(img, tuple(map(int, kp)), 2, (255, 0, 255), -1)
    
    return img


# ----------------------------------------------------------
# Load Models
# ----------------------------------------------------------
logger.info("Loading models...")
try:
    widerface_model = dg.load_model(
        model_name=widerface_model_name, inference_host_address=inference_host_address,
        zoo_url=zoo_url, token=token, device_type=device_type)
    logger.info("Loaded WiderFace model")

    face_embed_model = dg.load_model(
        model_name=face_embed_model_name, inference_host_address=inference_host_address,
        zoo_url=zoo_url, token=token, device_type=device_type)
    logger.info("Loaded embedding model")

    age_model = dg.load_model(
        model_name=age_model_name, inference_host_address=inference_host_address,
        zoo_url=zoo_url, token=token, device_type=device_type)
    logger.info("Loaded age model")

    gender_model = dg.load_model(
        model_name=gender_model_name, inference_host_address=inference_host_address,
        zoo_url=zoo_url, token=token, device_type=device_type)
    logger.info("Loaded gender model")

    emotion_model = dg.load_model(
        model_name=emotion_model_name, inference_host_address=inference_host_address,
        zoo_url=zoo_url, token=token, device_type=device_type)
    logger.info("Loaded emotion model")

except Exception:
    logger.exception("Failed to load models")
    raise


# ----------------------------------------------------------
# Initialize Components
# ----------------------------------------------------------
camera = ThreadedCamera(fps=CAMERA_FPS, width=PREVIEW_WIDTH, height=PREVIEW_HEIGHT)
tracker = ViewerTracker(iou_thr=0.25, emb_thr=0.5, timeout=3.0)
gaze_mgr = GazeSessionManager()
demo_worker = DemographicsWorker(age_model, gender_model, emotion_model, queue_size=DEMO_QUEUE_SIZE)

display_available = preview_camera
if preview_camera:
    try:
        cv2.namedWindow("test", cv2.WINDOW_NORMAL)
        cv2.destroyWindow("test")
        logger.info("Display available")
    except cv2.error:
        logger.warning("No display — preview disabled")
        display_available = False

fps_q = deque(maxlen=30)
last_fps_t = time.time()
last_log_t = time.time()
frame_counter = 0
total_faces_detected = 0
total_gaze_events = 0

logger.info("=" * 60)
logger.info("GAZE TRACKING v3.2.0-PRODUCTION STARTED")
logger.info(f"  Camera: {PREVIEW_WIDTH}x{PREVIEW_HEIGHT} @ {CAMERA_FPS}fps")
logger.info(f"  Skip frames: {SKIP_FRAMES} | Max faces: {MAX_FACES_PER_FRAME}")
logger.info(f"  Min loop time: {MIN_LOOP_TIME*1000:.0f}ms")
logger.info(f"  Thermal: throttle={THERMAL_THROTTLE_TEMP}°C critical={THERMAL_CRITICAL_TEMP}°C")
logger.info(f"  Gaze: Yaw ±{GAZE_YAW_THRESHOLD}° Pitch ±{GAZE_PITCH_THRESHOLD}° Min {MIN_GAZE_DURATION}s")
logger.info(f"  Gaze Debug: {'ENABLED' if GAZE_DEBUG else 'DISABLED'}")
logger.info(f"  Log file: {os.path.abspath(LOG_FILE)}")
logger.info("=" * 60)


# ----------------------------------------------------------
# Main Loop
# ----------------------------------------------------------
try:
    while True:
        loop_start = time.time()

        frame = camera.read()
        if frame is None:
            time.sleep(0.01)
            continue

        frame_counter += 1
        current_time = time.time()
        faces = []
        was_processed = False

        dt = current_time - last_fps_t
        fps_q.append(1.0 / dt if dt > 0 else 0)
        last_fps_t = current_time
        avg_fps = sum(fps_q) / len(fps_q) if fps_q else 0

        # ---- INFERENCE ----
        if frame_counter % SKIP_FRAMES == 0:
            cpu_temp = get_cpu_temp()

            if cpu_temp >= THERMAL_CRITICAL_TEMP:
                logger.warning(f"CRITICAL TEMP {cpu_temp:.0f}°C — sleeping 1s")
                time.sleep(1.0)
            elif cpu_temp >= THERMAL_THROTTLE_TEMP:
                logger.info(f"THROTTLE {cpu_temp:.0f}°C — sleeping 200ms")
                time.sleep(0.2)

            if cpu_temp < THERMAL_CRITICAL_TEMP:
                try:
                    det = widerface_model.predict(frame)
                    was_processed = True

                    if hasattr(det, 'results'):
                        det_items = []
                        for d in det.results:
                            try:
                                bbox = d.get("bbox") if isinstance(d, dict) else getattr(d, "bbox", None)
                                if bbox:
                                    x1, y1, x2, y2 = bbox
                                    det_items.append(((x2-x1)*(y2-y1), d))
                            except Exception:
                                continue
                        det_items.sort(key=lambda x: x[0], reverse=True)

                        for _, d in det_items[:MAX_FACES_PER_FRAME]:
                            try:
                                bbox = d.get("bbox") if isinstance(d, dict) else getattr(d, "bbox", None)
                                x1, y1, x2, y2 = map(int, bbox)
                                x1, y1 = max(0, x1), max(0, y1)
                                x2, y2 = min(frame.shape[1], x2), min(frame.shape[0], y2)
                                if x2 <= x1 or y2 <= y1:
                                    continue

                                crop = frame[y1:y2, x1:x2]
                                if isinstance(d, dict):
                                    kr = d.get("kpts") or d.get("landmarks") or d.get("keypoints")
                                else:
                                    kr = getattr(d, "kpts", None) or getattr(d, "landmarks", None) or getattr(d, "keypoints", None)
                                kpts = parse_keypoints(kr)

                                emb_raw = {}
                                if crop.size > 0:
                                    try:
                                        er = face_embed_model.predict(crop)
                                        if hasattr(er, 'results') and er.results:
                                            emb_raw = er.results[0]
                                    except Exception:
                                        pass

                                faces.append({
                                    "bbox": bbox, "kpts": kpts, "embedding": emb_raw,
                                    "crop": crop,
                                    "age_est": 0, "gender": "", "gender_score": 0.0,
                                    "emotion": "", "emotion_score": 0.0,
                                })
                            except Exception:
                                continue

                    total_faces_detected += len(faces)

                except Exception:
                    logger.exception("Detection failed")

        # ---- TRACKING + GAZE ----
        if was_processed and faces:
            bboxes = [f["bbox"] for f in faces]
            embs = [extract_embedding(f.get("embedding", {})) for f in faces]
            assignments = tracker.update(bboxes, embs)

            if len(assignments) != len(faces):
                logger.warning(f"Track mismatch: {len(assignments)} vs {len(faces)}")
            else:
                for i, f in enumerate(faces):
                    try:
                        vid, is_new = assignments[i]

                        if is_new:
                            crop = f.get("crop")
                            if crop is not None and crop.size > 0:
                                demo_worker.submit(vid, crop)

                        f.pop("crop", None)

                        kpts = f.get("kpts")
                        bbox = f.get("bbox")
                        
                        # Gaze detection
                        if kpts is not None and isinstance(kpts, np.ndarray) and kpts.shape == (5, 2):
                            gazing, yaw, pitch = is_gazing_at_screen(kpts, bbox)
                        else:
                            gazing, yaw, pitch = is_gazing_at_screen(None, bbox)

                        # FIXED: Use is_new_gaze flag
                        total_g, cont_g, is_new_gaze = gaze_mgr.update(vid, gazing, current_time)

                        f["is_gazing"] = gazing
                        f["gaze_duration"] = cont_g
                        f["total_gaze"] = total_g
                        f["yaw"] = yaw
                        f["pitch"] = pitch
                        f["viewer_id"] = vid

                        if vid not in viewer_profiles or is_new:
                            viewer_profiles[vid] = {
                                'age': 0, 'gender': '', 'first_seen': current_time, 'emotions': {}
                            }

                        prof = viewer_profiles[vid]
                        if prof['age'] > 0:
                            f["age_est"] = prof['age']
                        if prof['gender']:
                            f["gender"] = prof['gender']

                        emo = f.get("emotion", "")
                        if not emo and prof.get('emotions'):
                            emo = max(prof['emotions'].items(), key=lambda x: x[1])[0]
                            f["emotion"] = emo

                        # FIXED: Log only for NEW gaze starts
                        if is_new_gaze:
                            total_gaze_events += 1
                            log_gaze_event({
                                'timestamp': datetime.utcnow().isoformat() + "Z",
                                'event': 'gaze_start',
                                'viewer_id': vid,
                                'demographics': {'age': prof['age'], 'gender': prof['gender'], 'emotion': emo},
                                'head_pose': {
                                    'yaw': round(yaw, 2) if yaw is not None else None,
                                    'pitch': round(pitch, 2) if pitch is not None else None
                                }
                            })
                            logger.info(f"GAZE START: {vid} ({prof['gender']} {prof['age']}y)")

                    except Exception:
                        logger.debug(f"Face {i} error", exc_info=True)
                        continue

        # ---- HEARTBEAT ----
        if current_time - last_log_t >= LOG_INTERVAL:
            active = []
            for vid, s in gaze_mgr.sessions.items():
                if s['start'] is not None:
                    ct = current_time - s['start']
                    if ct >= MIN_GAZE_DURATION:
                        p = viewer_profiles.get(vid, {})
                        emos = p.get('emotions', {})
                        active.append({
                            'viewer_id': vid,
                            'continuous_gaze': round(ct, 1),
                            'total_gaze': round(s['total'], 1),
                            'demographics': {
                                'age': p.get('age', 0),
                                'gender': p.get('gender', ''),
                                'emotion': max(emos.items(), key=lambda x: x[1])[0] if emos else ''
                            }
                        })

            env = read_bme688_data()
            noise_db = read_noise_level_db()
            if noise_db is not None:
                env["noise_db"] = noise_db

            heartbeat = {
                'timestamp': datetime.utcnow().isoformat() + "Z",
                'event': 'heartbeat',
                'active_gazers': len(active),
                'tracked_viewers': len(gaze_mgr.sessions),
                'total_faces_detected': total_faces_detected,
                'total_gaze_events': total_gaze_events,
                'fps': round(avg_fps, 1),
                'cpu_temp': round(get_cpu_temp(), 1),
                'environment': env
            }
            if active:
                heartbeat['gazers'] = active
            
            log_gaze_event(heartbeat)
            logger.info(f"HEARTBEAT: gazers={len(active)} tracked={len(gaze_mgr.sessions)} "
                       f"faces_total={total_faces_detected} gaze_events={total_gaze_events} "
                       f"FPS={avg_fps:.1f} CPU={get_cpu_temp():.0f}°C")

            last_log_t = current_time

        # ---- CLEANUP STALE SESSIONS ----
        for ss in gaze_mgr.cleanup_stale():
            vid = ss['viewer_id']
            p = viewer_profiles.get(vid, {})
            log_gaze_event({
                'timestamp': datetime.utcnow().isoformat() + "Z",
                'event': 'session_end',
                'viewer_id': vid,
                'session_stats': ss,
                'demographics': {
                    'age': p.get('age', 0),
                    'gender': p.get('gender', ''),
                    'emotions': p.get('emotions', {})
                },
                'environment': read_bme688_data()
            })
            logger.info(f"SESSION END: {vid} gaze={ss['total_gaze_time']:.1f}s "
                       f"dur={ss['session_duration']:.1f}s "
                       f"engage={ss['engagement_rate']:.1%}")
            viewer_profiles.pop(vid, None)

        # ---- PREVIEW ----
        if display_available:
            disp = frame.copy()
            if faces:
                disp = draw_overlay(disp, faces)
            n_gaze = sum(1 for f in faces if f.get("is_gazing"))
            t = get_cpu_temp()
            cv2.putText(disp, f"FPS:{avg_fps:.1f} F:{len(faces)} G:{n_gaze} T:{t:.0f}C",
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.imshow("Audience Gaze Analysis", disp)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                logger.info("User exit")
                break

        # ---- ENFORCE MIN LOOP TIME ----
        elapsed = time.time() - loop_start
        if elapsed < MIN_LOOP_TIME:
            time.sleep(MIN_LOOP_TIME - elapsed)

except KeyboardInterrupt:
    logger.info("Interrupted")
finally:
    logger.info("Shutting down...")
    for vid in list(gaze_mgr.sessions.keys()):
        st = gaze_mgr.end_session(vid)
        if st:
            p = viewer_profiles.get(vid, {})
            log_gaze_event({
                'timestamp': datetime.utcnow().isoformat() + "Z",
                'event': 'shutdown_session_end',
                'viewer_id': vid,
                'session_stats': st,
                'demographics': {
                    'age': p.get('age', 0),
                    'gender': p.get('gender', ''),
                    'emotions': p.get('emotions', {})
                }
            })
            logger.info(f"Final: {vid} gaze={st['total_gaze_time']:.1f}s dur={st['session_duration']:.1f}s engage={st['engagement_rate']:.1%}")

    demo_worker.stop()
    camera.stop()
    if display_available:
        cv2.destroyAllWindows()
    logger.info(f"Clean shutdown. Total faces={total_faces_detected} gaze_events={total_gaze_events}")

2026-02-07 21:59:57,976 [INFO] === Log initialized: /home/william/ai_digital_signage/logs/audience_analysis_live.log ===
2026-02-07 21:59:57,981 [INFO] Log file size: 275848 bytes
2026-02-07 21:59:58,055 [INFO] BME688 sensor initialized (secondary)
2026-02-07 21:59:58,064 [INFO] Loading models...
2026-02-07 21:59:58,211 [INFO] Loaded WiderFace model
2026-02-07 21:59:58,271 [INFO] Loaded embedding model
2026-02-07 21:59:58,335 [INFO] Loaded age model
2026-02-07 21:59:58,395 [INFO] Loaded gender model
2026-02-07 21:59:58,450 [INFO] Loaded emotion model


[4:07:01.380816755] [67400]  INFO Camera camera_manager.cpp:330 libcamera v0.5.2+99-bfd68f78
[4:07:01.388304402] [67493]  INFO RPI pisp.cpp:720 libpisp version v1.2.1 981977ff21f3 29-04-2025 (14:13:50)
[4:07:01.390981118] [67493]  INFO IPAProxy ipa_proxy.cpp:180 Using tuning file /usr/share/libcamera/ipa/rpi/pisp/imx708.json
[4:07:01.398474561] [67493]  INFO Camera camera_manager.cpp:220 Adding camera '/base/axi/pcie@1000120000/rp1/i2c@88000/imx708@1a' for pipeline handler rpi/pisp
[4:07:01.398496321] [67493]  INFO RPI pisp.cpp:1179 Registered camera /base/axi/pcie@1000120000/rp1/i2c@88000/imx708@1a to CFE device /dev/media0 and ISP device /dev/media2 using PiSP variant BCM2712_D0
[4:07:01.402484580] [67400]  INFO Camera camera.cpp:1215 configuring streams: (0) 640x480-RGB888/sRGB (1) 1536x864-BGGR_PISP_COMP1/RAW
[4:07:01.402591710] [67493]  INFO RPI pisp.cpp:1483 Sensor: /base/axi/pcie@1000120000/rp1/i2c@88000/imx708@1a - Selected sensor format: 1536x864-SBGGR10_1X10/RAW - Selected CF

2026-02-07 21:59:59,505 [INFO] Threaded camera started at 640x480 @ 10fps
2026-02-07 21:59:59,510 [INFO] Demographics worker started (queue_size=8)
2026-02-07 21:59:59,665 [INFO] Display available
2026-02-07 21:59:59,672 [INFO] ============================================================
2026-02-07 21:59:59,680 [INFO] GAZE TRACKING v3.2.0-PRODUCTION STARTED
2026-02-07 21:59:59,687 [INFO]   Camera: 640x480 @ 10fps
2026-02-07 21:59:59,692 [INFO]   Skip frames: 3 | Max faces: 3
2026-02-07 21:59:59,700 [INFO]   Min loop time: 100ms
2026-02-07 21:59:59,706 [INFO]   Thermal: throttle=78.0°C critical=82.0°C
2026-02-07 21:59:59,714 [INFO]   Gaze: Yaw ±30° Pitch ±20° Min 0.5s
2026-02-07 21:59:59,721 [INFO]   Gaze Debug: DISABLED
2026-02-07 21:59:59,727 [INFO]   Log file: /home/william/ai_digital_signage/logs/audience_analysis_live.log
2026-02-07 21:59:59,732 [INFO] ============================================================
2026-02-07 22:00:05,117 [INFO] GAZE_EVENT: {"timestamp": "2026-02-07T1